In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import gc
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Metric

In [ ]:
train_label_df = pd.read_csv('../input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')
test_label_df = pd.read_csv('../input/planet-understanding-the-amazon-from-space/sample_submission_v2.csv/sample_submission_v2.csv')


In [ ]:
test_label_df

In [ ]:
train_name = train_label_df['image_name'].values
gc.collect()

In [ ]:
def collect_image(jpg_folder,pic_names,size=None):
    tif = []
    for l in pic_names:
        img = cv2.imread(f'../input/planets-dataset/planet/planet/{jpg_folder}/{l}.jpg',cv2.COLOR_BGR2RGB)
        if size == None:
            img = img
        else: 
            img = cv2.resize(img,(size,size))
        img = img/255
        tif.append(img)
    tif = np.array(tif)
    gc.collect()
    return tif  

In [ ]:
x = collect_image('train-jpg',train_name,size=48)
gc.collect()

In [ ]:

print(x.shape)
del train_name
gc.collect()

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(x[4])
plt.title(f'the title is{train_label_df.tags[4]}')
plt.show()
gc.collect()
          

In [ ]:
label_list = []
for tag_str in train_label_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)
for label in label_list:
    train_label_df[label] = train_label_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
gc.collect()       

In [ ]:
label_list

In [ ]:
y = train_label_df.drop(columns=['tags','image_name'])
del train_label_df
del test_label_df
gc.collect()

In [ ]:
no_labels = len(label_list)
y = y.values
print(no_labels)
print(y.shape)

In [ ]:
data_gen = ImageDataGenerator(
rotation_range=30,
zoom_range=0.2,
width_shift_range=0.1,
height_shift_range=0.1,
horizontal_flip=True)
data_gen.fit(x)

In [ ]:
print(x.shape)
x_train,x_eval,y_train,y_eval = train_test_split(x,y,test_size=0.2,random_state=42)
del x
del y
print(x_train.shape)
gc.collect()

In [ ]:
gc.collect()
model = Sequential()
model.add(Conv2D(32,(3,3),padding="same",activation="relu",input_shape=(48,48,3)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(no_labels,activation='sigmoid'))
model.compile(optimizer='adam',loss=tf.keras.losses.binary_crossentropy, metrics=['accuracy'])
history = model.fit(x_train,y_train,batch_size=1000,epochs=5,validation_data=(x_eval,y_eval))
gc.collect()

In [ ]:
plt.figure()
plt.plot(history.history['loss'],'blue')
plt.plot(history.history['val_loss'],'red')
plt.legend(['training loss','validation loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show


In [ ]:
plt.figure()
plt.plot(history.history['loss'],'blue')
plt.plot(history.history['val_loss'],'red')
plt.legend(['training loss','validation loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.show


In [ ]:
fbeta = model.evaluate(x_eval,y_eval)
print(fbeta)
del x_eval
del y_eval
del x_train
del y_train
del fbeta
gc.collect()

In [ ]:
import os
x_test = []
path = '../input/planets-dataset/planet/planet/test-jpg/'
for l in os.listdir(path):
    img = cv2.imread(os.path.join(path,l))
    img = cv2.resize(img,(48,48))
    img = img/255
    x_test.append(img)
dir_= '../input/planets-dataset/test-jpg-additional/test-jpg-additional/'
for t in os.listdir(dir_):
    img = cv2.imread(os.path.join(dir_,t))
    img = cv2.resize(img,(48,48))
    img = img/255
    x_test.append(img)
x_test = np.asarray(x_test)
print(x_test.shape)
gc.collect()

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(x_test[0])
# plt.title(f'the title is{train_label_df.tags[1]}')
plt.show()
gc.collect()

In [ ]:

pred = np.around(model.predict(x_test))
# del x_test
gc.collect()

In [ ]:
label = np.array(label_list)
print(pred[10])

print(label[np.argwhere(pred[10])])# correct way of predicting
print(label)


In [ ]:
print(len(pred))
pred.shape

In [ ]:
del x_test
tag = []
for no in range(len(pred)): 
    tags = label[np.argwhere(pred[no])]
    tags = tags.reshape(-1)
    tags = tags.tolist()
    tags = ' '.join(tags)
    tag.append(tags)
gc.collect()
    

In [ ]:
image = []
for l in os.listdir(path):
    image.append(l)
for t in os.listdir(dir_):
    image.append(t)

In [ ]:
# image[1]
data = {'image_name':image ,'tags':tag}
submission_file = pd.DataFrame().from_dict(data)
submission_file

In [ ]:
submission_file.to_csv('submission.csv')
